In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("data/Divvy_Trips_2022_full.csv.gz", compression="gzip")

# Общая информация о датасете

In [3]:
df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,C2F7DD78E82EC875,electric_bike,2022-01-13 11:59:47,2022-01-13 12:02:44,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012800,-87.665906,42.012560,-87.674367,casual
1,A6CF8980A652D272,electric_bike,2022-01-10 08:41:56,2022-01-10 08:46:17,Glenwood Ave & Touhy Ave,525,Clark St & Touhy Ave,RP-007,42.012763,-87.665967,42.012560,-87.674367,casual
2,BD0F91DFF741C66D,classic_bike,2022-01-25 04:53:40,2022-01-25 04:58:01,Sheffield Ave & Fullerton Ave,TA1306000016,Greenview Ave & Fullerton Ave,TA1307000001,41.925602,-87.653708,41.925330,-87.665800,member
3,CBB80ED419105406,classic_bike,2022-01-04 00:18:04,2022-01-04 00:33:00,Clark St & Bryn Mawr Ave,KA1504000151,Paulina St & Montrose Ave,TA1309000021,41.983593,-87.669154,41.961507,-87.671387,casual
4,DDC963BFDDA51EEA,classic_bike,2022-01-20 01:31:10,2022-01-20 01:37:12,Michigan Ave & Jackson Blvd,TA1309000002,State St & Randolph St,TA1305000029,41.877850,-87.624080,41.884621,-87.627834,member


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5667717 entries, 0 to 5667716
Data columns (total 13 columns):
 #   Column              Dtype  
---  ------              -----  
 0   ride_id             object 
 1   rideable_type       object 
 2   started_at          object 
 3   ended_at            object 
 4   start_station_name  object 
 5   start_station_id    object 
 6   end_station_name    object 
 7   end_station_id      object 
 8   start_lat           float64
 9   start_lng           float64
 10  end_lat             float64
 11  end_lng             float64
 12  member_casual       object 
dtypes: float64(4), object(9)
memory usage: 562.1+ MB


## Работа с данными

In [26]:
# Данные станций
station = df[["start_station_name", "started_at", "member_casual"]]
station = station.rename(columns={"start_station_name": "station_name"})

In [25]:
station.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5667717 entries, 0 to 5667716
Data columns (total 3 columns):
 #   Column         Dtype 
---  ------         ----- 
 0   station_name   object
 1   started_at     object
 2   member_casual  object
dtypes: object(3)
memory usage: 129.7+ MB


In [28]:
station["started_at"] = station["started_at"].astype('datetime64[D]')

In [29]:
day = station.groupby(["started_at"]).count()

In [30]:
day.rename(columns={"station_name":"count"}, inplace=True)

In [31]:
sorted_day = day.sort_values(by='count', ascending=False)
sorted_day.drop('member_casual', axis= 1 , inplace= True ) 

In [32]:
sorted_day

,count
started_at,
2022-07-09,31364
2022-06-18,30526
2022-04-23,30267
2022-05-29,29828
2022-07-30,28606
...,...
2022-12-26,990
2022-02-02,543
2022-12-25,406


In [33]:
ride = station[(station['started_at'] >= "2022-08-07") & (station['started_at'] < "2022-08-08")]

In [34]:
ride = ride.groupby(["started_at", "member_casual"]).count()

In [35]:
ride = ride.reset_index(["started_at", "member_casual"])

In [36]:
ride.rename(columns={"station_name":"count"}, inplace=True)

In [37]:
ride

,started_at,member_casual,count
0,2022-08-07,casual,7227
1,2022-08-07,member,6955


In [38]:
member = int(ride[ride["member_casual"] == "member"]["count"])
casual = int(ride[ride["member_casual"] == "casual"]["count"])

In [40]:
people = member // 2 + casual
people

10704

In [41]:
member = int(people * 0.74)
casual = int(people * 0.26)
print(member, casual)

7920 2783
